# Finite volume discretization of unsteady two-dimensional advection-diffusion equation

The equation we would like to solve numerically is the following:
$$\rho \frac{\partial \phi}{\partial t} + \rho u_j \frac{\partial \phi}{\partial x_j} = \frac{\partial}{\partial x_j} \biggl(\Gamma \frac{\partial\phi}{\partial x_j}\biggr) + S $$

In two-dimensions the partial derivatives are expanded as:
$$\rho \frac{\partial \phi}{\partial t} + \rho u_x \frac{\partial \phi}{\partial x} + \rho u_y \frac{\partial \phi}{\partial y} = \frac{\partial}{\partial x} \biggl(\Gamma \frac{\partial\phi}{\partial x}\biggr) +\frac{\partial}{\partial y} \biggl(\Gamma \frac{\partial\phi}{\partial y}\biggr) + S $$


In what follows the source term is taken to be zero $S=0$:
$$\rho \frac{\partial \phi}{\partial t} + \rho u_x \frac{\partial \phi}{\partial x} + \rho u_y \frac{\partial \phi}{\partial y} = \frac{\partial}{\partial x} \biggl(\Gamma \frac{\partial\phi}{\partial x}\biggr) +\frac{\partial}{\partial y} \biggl(\Gamma \frac{\partial\phi}{\partial y}\biggr) $$

## Unsteady purely conduction equation

We will start with unsteady purely conduction equation:
$$\rho \frac{\partial \phi}{\partial t} = \frac{\partial}{\partial x}\biggl(\Gamma \frac{\partial \phi}{\partial x}\biggr) + \frac{\partial}{\partial y}\biggl(\Gamma \frac{\partial \phi}{\partial y}\biggr)$$

Let's integrate this equation over the control volume:
$$\rho \int_w^e\int_s^n \frac{\partial \phi}{\partial t} dx dy = \int_w^e\int_s^n \frac{\partial}{\partial x} \biggl(\Gamma \frac{\partial \phi}{\partial x}\biggr) dx dy + \int_w^e\int_s^n \frac{\partial}{\partial y} \biggl(\Gamma \frac{\partial \phi}{\partial y}\biggr) dx dy$$

![alt text](integration.png "2D control volume")

The integration leads to the following:
$$\rho \Delta x_P \Delta y_P \frac{\partial}{\partial t} \phi_P   = \Delta y_P \biggl(\Gamma_e \Bigl(\frac{\partial\phi}{\partial x}\Bigr)_e-\Gamma_w \Bigl(\frac{\partial\phi}{\partial y}\Bigr)_w\biggr) + \Delta x_P \biggl(\Gamma_n \Bigl(\frac{\partial\phi}{\partial y}\Bigr)_n-\Gamma_s \Bigl(\frac{\partial\phi}{\partial y}\Bigr)_s\biggr)$$

Assume that the partial derivatives can be represented through forward difference:
$$\Bigl(\frac{\partial \phi}{\partial x}\Bigr)_e = \frac{\phi_E - \phi_P}{\Delta x_e}$$
$$\Bigl(\frac{\partial \phi}{\partial x}\Bigr)_w = \frac{\phi_P - \phi_W}{\Delta x_w}$$
$$\Bigl(\frac{\partial \phi}{\partial y}\Bigr)_n = \frac{\phi_N - \phi_P}{\Delta y_n}$$
$$\Bigl(\frac{\partial \phi}{\partial y}\Bigr)_s = \frac{\phi_P - \phi_S}{\Delta y_s}$$,
where $\Delta x_P = 0.5(\Delta x_e + \Delta x_w)$ and $\Delta y_P = 0.5(\Delta y_n + \Delta y_s)$


Thus, after substitution in the numerical equation the following is obtained:
$$\rho \Delta x_P \Delta y_P \frac{\partial}{\partial t}\phi_P = \Delta y_P \biggl(\Gamma_e \frac{\phi_E - \phi_P}{\Delta x_e} - \Gamma_w \frac{\phi_P-\phi_W}{\Delta x_w}\biggr)+\Delta x_P \biggl(\Gamma_n \frac{\phi_N-\phi_P}{\Delta y_n}-\Gamma_s \frac{\phi_P - \phi_S}{\Delta y_s}\biggr)$$

Let's combine the terms accordingly:
$$\rho \Delta x_P \Delta y_P \frac{\partial}{\partial t}\phi_P = \phi_P \biggl(-\Gamma_e\frac{\Delta y_P}{\Delta x_e}-\Gamma_w \frac{\Delta y_P}{\Delta x_w}-\Gamma_n \frac{\Delta x_P}{\Delta y_n} - \Gamma_s \frac{\Delta x_P}{\Delta y_s}\biggr) + \phi_E \Gamma_e \frac{\Delta y_P}{\Delta x_e} + \phi_W \Gamma_w \frac{\Delta y_P}{\Delta x_w} + \phi_S \Gamma_s \frac{\Delta x_P}{\Delta y_s} + \phi_N \Gamma_n \frac{\Delta x_P}{\Delta y_n}$$

Now the problem which we would like to solve is a slab with two constraints $\phi(x=0,y)=1$ and $\phi(x=L,y)=0$. As this problem is two-dimensional, we will impose two adiabatic boundary conditions on top and the bottom of the domain, i.e. $\frac{\partial \phi(x,y=0)}{\partial y}=0$ and $\frac{\partial \phi(x,y=H)}{\partial y}=0$. 

To derive boundary conditions we need to introduce the control volume near boundaries. The points are placed in the center of control volumes, so the boundary conditions are naturally involved.

![alt text](boundary_volumes.png?arg "Boundary control volumes")

Now, let's try to compose the matrix assuming the following quantities: $\rho = 1$, $\Gamma=1$, $L=1$, $H=2$. The numerical domain for points will be taken as $0..NX+1$ and $0..NY+1$, where $NX=10$ and $NY=20$. Then we will have $\Delta x = \Delta y = 0.1$. Notice that points close to boundaries will have only half distance of $0.5 \Delta x$ and $0.5 \Delta y$.

The matrix will be is as follows:
$$\begin{aligned}
&\phi_{0,0..NY+1}=0\\
&\phi_{NX+1,0..NY+1}=1\\
&\frac{\partial \phi_{1..NX,0}}{\partial y} = 0\\
&\frac{\partial \phi_{1..NX,NY+1}}{\partial y} = 0
\end{aligned}$$

For the bulk nodes the following equation will be fulfilled assuming $\rho$, $\Gamma$, $L$ and $H$ from above:
$$
\begin{aligned}
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{i,j} = -4 \phi_{i,j} +  \phi_{i-1,j} + \phi_{i+1,j} +  \phi_{i,j-1} + \phi_{i,j+1} &&\text{for }i=2..NX-1\text{ and }j=2..NY-1\\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{i,1} = -3 \phi_{i,1} + \phi_{i-1,1} + \phi_{i+1,1} + \phi_{i,2} &&\text{for }i=2..NX-1\\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{i,NY} = -3 \phi_{i,NY} + \phi_{i-1,NY} + \phi_{i+1,NY} +  \phi_{i,NY-1} &&\text{for }i=2..NX-1\\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{1,j} = -5 \phi_{1,j} + 2\phi_{0,j} + \phi_{2,j} +  \phi_{1,j-1}+\phi_{1,j+1} &&\text{for }j=2..NY-1\\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{NX,j} = -5 \phi_{NX,j} + \phi_{NX-1,j} + 2\phi_{NX+1,j} +  \phi_{NX,j-1}+\phi_{NX,j+1} &&\text{for }j=2..NY-1\\
\end{aligned}$$

The corner BCs are indicated below:
$$
\begin{aligned}
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{1,1} = -4 \phi_{1,1} +  2 \phi_{0,1} + \phi_{2,1} +  \phi_{1,2} \\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{1,NY} = -4 \phi_{1,NY} + 2\phi_{0,NY} + \phi_{1,NY-1} + \phi_{2,NY} \\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{NX,1} = -4 \phi_{NX,1} + 2\phi_{NX+1,1} + \phi_{NX-1,1} +  \phi_{NX,2} \\
&\Delta x \Delta y \frac{\partial}{\partial t}\phi_{NX,NY} = -4 \phi_{NX,NY} + 2\phi_{NX+1,NY} + \phi_{NX-1,NY} +  \phi_{NX,NY-1}\\
\end{aligned}
$$

Though the system looks quite simple, the connection between coefficients between rows and columns doesn't give an opportunity to easily write the matrix. Instead, we will start numerically allocate the matrix:

In [1]:
import numpy
import pylab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['pylab']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
NX=10
NY=20
rhs = numpy.zeros(((NY+2)*(NX+2),))
mat = numpy.zeros(((NX+2)*(NY+2),(NX+2)*(NY+2)))

xfv = numpy.zeros((NX+2,))
xfv[NX+1]=1.0
xfv[1:NX+1] = numpy.linspace(0.5/NX,1-0.5/NX,NX)
yfv = numpy.zeros((NY+2,))
yfv[NY+1] = 2.0
yfv[1:NY+1] = numpy.linspace(1/NY,2 - 1/NY,NY)

Notice that boundary conditions will enter as follows:

In [6]:
def convert(indX,indY):
    return indY*(NX+2)+indX

for j in range(NY+2):
    rhs[convert(0,j)] = 1
    rhs[convert(NX+1,j)] = 0
    mat[convert(0,j),convert(0,j)] = 1
    mat[convert(NX+1,j),convert(NX+1,j)] = 1
    
for i in range(1,NX+1):
    mat[convert(i,0),convert(i,0)]=1
    mat[convert(i,0),convert(i,1)]=-1
    mat[convert(i,NY+1),convert(i,NY+1)]=1
    mat[convert(i,NY+1),convert(i,NY)]=-1

Let's put other conditions into the matrix:

In [4]:
for i in range(2,NX):
    mat[convert(i,1),convert(i,1)]=-3
    mat[convert(i,1),convert(i-1,1)]=1
    mat[convert(i,1),convert(i+1,1)]=1
    mat[convert(i,1),convert(i,2)]=1
    
    mat[convert(i,NY),convert(i,1)]=-3
    mat[convert(i,NY),convert(i-1,1)]=1
    mat[convert(i,NY),convert(i+1,1)]=1
    mat[convert(i,NY),convert(i,2)]=1
    
    for j in range(2,NY):
        ind_center = j*(NX+2)+i
        ind_north  = ind_center+NX+2
        ind_south  = ind_center-NX-2
        ind_west   = ind_center-1
        ind_east   = ind_center+1
        mat[ind_center,ind_center] = -4
        mat[ind_center,ind_north]  = 1
        mat[ind_center,ind_south]  = 1
        mat[ind_center,ind_west]   = 1
        mat[ind_center,ind_east]   = 1
for i in range(1,NX+1):
    ind1_center= 1*(NX+2)+i
    ind1_north = ind1_center+NX+2
    ind1_west  = ind1_center-1
    ind1_east  = ind1_center+1
    ind2_center= NY*(NX+2)+i
    ind2_south = ind2_center-NX-2
    ind2_west  = ind2_center-1
    ind2_east  = ind2_center+1
    
    mat[ind1_center,ind1_center]=-3
    mat[ind1_center,ind1_north] =1
    mat[ind1_center,ind1_west]  =1
    mat[ind1_center,ind1_east]  =1
    
    mat[ind2_center,ind2_center]=-3
    mat[ind2_center,ind2_south] =1
    mat[ind2_center,ind2_east]  =1
    mat[ind2_center,ind2_west]  =1

Withouth discretization of time (basically assuming the steady-state condition), let's try to see if we can obtain the linear profile.

In [5]:
phi = numpy.linalg.solve(mat,rhs)
phi_mat=phi.reshape((NY+2,NX+2))

LinAlgError: Singular matrix

In [ ]:
pylab.imshow(phi_mat)

In [ ]:
pylab.plot(xfv,phi_mat[10,:])

Now let's integrate over the time. The general time integration is as follows:
$$\int_t^{t+\Delta t}{\phi_P dt} = (f \phi_P^{N+1} + (1-f)\phi_P^N)\Delta t$$ 
where $f$ is the implicitness parameter from 0 to 1, $\phi_P^{N+1}$ is the value at time $t+\Delta t$ and $\phi_P^N$ is the value at time $t$.

Two most popular discretizations are implicit discretization ($f=1$) and Crank-Nicolson discretization ($f=0.5$).

So the equations containing temporal parts are as follows:
$$
\begin{aligned}
&10^{-2} \frac{ \phi_{i,j}^{N+1}-\phi_{i,j}^N}{\Delta t} = f(-8 \phi_{i,j}^{N+1} + 2 \phi_{i-1,j}^{N+1} + 2 \phi_{i+1,j}^{N+1} + 2 \phi_{i,j-1}^{N+1} + 2 \phi_{i,j+1}^{N+1})+(1-f)(-8 \phi_{i,j}^N + 2 \phi_{i-1,j}^N + 2 \phi_{i+1,j}^N + 2 \phi_{i,j-1}^N + 2 \phi_{i,j+1}^N) &&\text{for }i=1..NX\text{ and }j=2..NY-1\\
&0.75\times 10^{-2} \frac{\phi_{i,1}^{N+1}-\phi_{i,1}^N}{\Delta t} = f (-6 \phi^{N+1}_{i,1} + 2 \phi^{N+1}_{i-1,1} + 2 \phi^{N+1}_{i+1,1} + 2 \phi^{N+1}_{i,2})+(1-f)(-6 \phi_{i,1}^N + 2 \phi_{i-1,1}^N + 2 \phi_{i+1,1}^N + 2 \phi_{i,2}^N) &&\text{for }i=1..NX\\
&0.75\times 10^{-2} \frac{\phi_{i,NY}^{N+1}-\phi_{i,NY}^N}{\Delta t} =  f (-6 \phi_{i,NY}^{N+1} + 2 \phi_{i-1,NY}^{N+1} + 2 \phi_{i+1,NY}^{N+1} + 2 \phi_{i,NY-1}^{N+1}) +(1-f) (-6 \phi_{i,NY}^N + 2 \phi_{i-1,NY}^N + 2 \phi_{i+1,NY}^N + 2 \phi_{i,NY-1}^N) &&\text{for }i=1..NX\\
\end{aligned}
$$

So we need to reconstruct the matrix and its BCs:

In [ ]:
# Parameters related to the time-stepping
f = 1.0
deltat = 0.1

def updateMatrix(mat,rhs):
    for i in range(1,NX+1):
        for j in range(2,NY):
            ind_center = j*(NX+2)+i
            ind_north  = ind_center+NX+2
            ind_south  = ind_center-NX-2
            ind_west   = ind_center-1
            ind_east   = ind_center+1
            mat[ind_center,ind_center] = 0.01/deltat + 8*f
            mat[ind_center,ind_north]  = -2*f
            mat[ind_center,ind_south]  = -2*f
            mat[ind_center,ind_west]   = -2*f
            mat[ind_center,ind_east]   = -2*f
            rhs[ind_center] = 0.01/deltat*phi_old[ind_center] + (1-f)*(-8*phi_old[ind_center] + 2*phi_old[ind_north] + 2*phi_old[ind_south]
                                                                       +2*phi_old[ind_west] + 2*phi_old[ind_east]) 

    for i in range(1,NX+1):
        ind1_center= 1*(NX+2)+i
        ind1_north = ind1_center+NX+2
        ind1_west  = ind1_center-1
        ind1_east  = ind1_center+1
        ind2_center= NY*(NX+2)+i
        ind2_south = ind2_center-NX-2
        ind2_west  = ind2_center-1
        ind2_east  = ind2_center+1

        mat[ind1_center,ind1_center]=0.0075/deltat + 6*f
        mat[ind1_center,ind1_north] =-2*f
        mat[ind1_center,ind1_west]  =-2*f
        mat[ind1_center,ind1_east]  =-2*f
        rhs[ind1_center] = 0.0075/deltat*phi_old[ind1_center]+(1-f)*(-6*phi_old[ind1_center]+2*phi_old[ind1_north]
                                                                     +2*phi_old[ind1_west]+2*phi_old[ind1_east])

        mat[ind2_center,ind2_center]=0.0075/deltat + 6*f
        mat[ind2_center,ind2_south] =-2*f
        mat[ind2_center,ind2_east]  =-2*f
        mat[ind2_center,ind2_west]  =-2*f
        rhs[ind2_center] = 0.0075/deltat*phi_old[ind2_center]+(1-f)*(-6*phi_old[ind2_center]+2*phi_old[ind2_south]
                                                                     +2*phi_old[ind2_west]+2*phi_old[ind2_east])   
        

Let's find the analytical solution for the problem:
$$
0.01 \partial_t \phi = \partial_x^2 \phi, \text{ with }\phi(x,t=0)=0, \phi(x=0) = 1, \phi(x=1)=0
$$

We actually know the solution for the following equation:
$$
\partial_t w = \alpha \partial_x^2 w, \text{ with } w(x,t=0)=f(x), w(x=0,t)=0, w(x=l,t)=0
$$
The solution is the following:
$$
\begin{aligned}
&w(x,t)=\sum_{k=1}^{\infty}{\beta_k \sin(\frac{k \pi}{l} x)\exp\bigl(-\alpha \frac{\pi^2 k^2 t}{l^2}\bigr)}\\
&\beta_k = \frac{2}{l}\int_0^l{f(x) \sin(\frac{k \pi}{l} x) dx}
\end{aligned}
$$
Let's separate the solution for $\phi$ into homogenous and particular parts:
$$
\phi = 1-x + \phi_h,
$$
where $\phi_h(x=0,t) = 0$, $\phi_h(x=1,t)=0$, $\phi_h(x,t=0)=x-1$.

Transfering this equation we see that $f(x)=x-1$, $l = 1$, $\alpha = 100$. The solution for homogenuous part is the following:
$$
\begin{aligned}
&\phi_h(x,t)=\sum_{k=1}^{\infty}{\beta_k \sin(k \pi x)\exp\bigl(-\alpha \pi^2 k^2 t\bigr)}\\
&\beta_k = 2\int_0^1{(x-1) \sin(k \pi x) dx}=-\frac{2}{k\pi}
\end{aligned}
$$
Then the homogenuous part is the following:
$$
\phi_h(x,t) = -\sum_{k=1}^{\infty}{\frac{2}{k\pi}\sin(k \pi x)\exp\bigl(-\alpha \pi^2 k^2 t\bigr)}
$$
Then the analytical solution is the following:
$$
\phi(x,t)=1-x-\sum_{k=1}^{\infty}{\frac{2}{k\pi}\sin(k \pi x)\exp\bigl(-\alpha \pi^2 k^2 t\bigr)}
$$

In [ ]:
def analytical(time):
    numTerms = 200
    numX = 101

    x=numpy.linspace(0,1,numX)
    sol = 1 - x
    alpha = 1
    for term in range(1,numTerms+1):
        sol = sol - 2.0/(term*numpy.pi)*numpy.sin(term * numpy.pi*x)*numpy.exp(-alpha*(numpy.pi**2)*(term**2)*time)
    return x,sol


In [ ]:
import time
from IPython import display
totalTime = 0.3
numIterations = int(ceil(totalTime/deltat))
print("Number of iterations",numIterations)

In [ ]:
phi=numpy.zeros(((NY+2)*(NX+2),))
phi[::NX+2]=1
for iteration in range(1,numIterations+1):
    phi_old = phi
    updateMatrix(mat,rhs)
    
    phi = numpy.linalg.solve(mat,rhs)
    phi_mat=phi.reshape((NY+2,NX+2))
    pylab.figure(1)
    pylab.imshow(phi_mat)
    print("Iteration = ",iteration)
    display.clear_output(wait=True)
    display.display(pylab.gcf())
    
    pylab.figure(2)
    xan,an = analytical(iteration*deltat)
    pylab.plot(xan,an)
    pylab.plot(xfv,phi_mat[10,:],"+")
    time.sleep(0.1)